***Three datasets are created.  
These Datasets are created after cleaning all the datasets provided  
All_PATIENTS.csv This Dataset contains all 25 patients data from HCPA000*P.csv  
DEMOGRAPHIC.csv This dataset contains demogrpahic data provided in "T1DM_patient_sleep_demographics_with_race.csv" PATINETS_WITH_DEMOGRAPHIC.csv This dataset is merge of above two datasets***
<pre><strong>
This Jupyter notebooks has cells:
    Importing modules  
    Declaring variables  
    A common function for Data Quality Report  
    A function for patients data set to generate a Data Quality Report  
    A function for demographic 
    data set to generate a Data Quality Report  
    A dataframe by merging all csv files having patient data  
    Change time into date and time and write the clean data into csv file  
    Dataframe for demographic data by renaming all column names to lowercase with underscore betweeen words and created csv file  
    Dataframe for merged allpatients and demographic data and created csv file  
    Data Quality report of demographic data set  
    Data Quality report of merged patients and demographic data sets  
</strong><pre>

In [21]:
import pandas as pd
import glob
import os

In [25]:
#Variables
path = "HUPA-UC Diabetes Dataset/"
demographic_path = path + "T1DM_patient_sleep_demographics_with_race.csv"
all_patients_path = path + "ALL_PATIENTS.csv"
modified_demographic_path = path+"DEMOGRAPHIC.csv"
patients_demographic_path = path +"PATIENTS_WITH_DEMOGRAPHIC.csv"

In [23]:
def data_quality_report_common(df):
    print("DATA QUALITY REPORT")
    print("="*50)
    
    # Shape
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    print("="*50)
    
    # Missing values
    print("*"*50)
    print("\n Check for Missing Values:")
    print("*"*50)
    print("\n Number of missing column values are:\n",df.isnull().sum(), " ")
    
    # Duplicates
    print("*"*50)
    print("\n Check for Duplicate Rows:")
    print("*"*50)
    print("\n Number of duplicate rows  are :", df.duplicated().sum())
    
    # Data types
    print("*"*50)
    print("\n Check for Column Data Types:")
    print("*"*50)
    print(df.dtypes)
    
    # Unique Patient IDs
    if "patient_id" in df.columns:
        print("*"*50)
        print("\n Check for Patient IDs uniqueness and number of records:")
        print("*"*50)
        print("Number of Unique Patient IDs:", df['patient_id'].nunique())
        print("Total patient records are  :", len(df))
    
    # Numeric ranges
    print("*"*50)
    print("\n Check for Numeric Column Summary:")
    print("*"*50)
    print(df.describe().T[['min', 'max', 'mean']])
    

In [24]:
 #Check for impossible or outlier values
def demographic_data_quality_report(df):
     #Check for impossible or outlier values
    print("*"*50)
    print("\n Check for Impossible or outlier values:")
    print("*"*50)
    print('Number of invalid age value (0-100)', ((demographic_df['age']<0) | (demographic_df['age']>100)).sum())
    print("Number of invalid Gender values not ['Male'], ['Female']", 
          ((demographic_df['gender']!='Male') & (demographic_df['gender']!='Female')).sum())
    print("Number of avg sleep hours greater than 24", (demographic_df['avg_sleep_duration_hrs']>24).sum())
    print("Number of invalid sleep quality score value(1-10)", 
          ((demographic_df['sleep_quality_score']<1) & (demographic_df['sleep_quality_score']>10)).sum())
    print("Number of invalid sleep disturbances percentage >100",(demographic_df['sleep_disturbances_percentage']>100).sum())
    print("="*50)


In [37]:
#Creating a dataframe by merging all csv files having patient data
csv_files = glob.glob(os.path.join(path,"H*.csv"))
df_list =[]
for file in csv_files:
    patientId = os.path.basename(file).split(".")[0]
    df = pd.read_csv(file,sep=";")
    df["patient_id"] = patientId
    df_list.append(df)
patients_df = pd.concat(df_list, ignore_index = True)
patients_df.columns

Index(['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate',
       'bolus_volume_delivered', 'carb_input', 'patient_id'],
      dtype='object')

In [27]:
#change time into date and time and write the clean data into csv file
#reason: 1.time is in String format. To analyze and manipulate time-series data, time should be in datetime format
#2. To filter data by date, extracting time compnents, and sorting efficiently
#patients_df["time"] = pd.to_datetime(patients_df["time"], errors="coerce")
#patients_df["time"].dt.day==13

patients_df.to_csv(all_patients_path,index=False)


In [28]:
#Renamed Column names as per the HUPA standred naming conventions and created new csv file
demographic_df = pd.read_csv(demographic_path)
demographic_df.rename(columns={"Patient_ID":"patient_id","Age":"age","Gender":"gender","Race":"race","Average Sleep Duration (hrs)":"avg_sleep_duration_hrs","Sleep Quality (1-10)":"sleep_quality_score","% with Sleep Disturbances":"sleep_disturbances_percentage"},inplace = True)
demographic_df.columns
demographic_df.to_csv(modified_demographic_path, index=False)


In [32]:
#Dataframe for merged allpatients and demographic data and created csv file
patients_demographic_df = pd.merge(patients_df,demographic_df, on ="patient_id", how="inner")
patients_demographic_df.to_csv(patients_demographic_path, index=False)
patients_demographic_df.columns


Index(['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate',
       'bolus_volume_delivered', 'carb_input', 'patient_id', 'age', 'gender',
       'race', 'avg_sleep_duration_hrs', 'sleep_quality_score',
       'sleep_disturbances_percentage'],
      dtype='object')

In [ ]:
# Data Quality Report

# This Python function generates a comprehensive data quality report for a given DataFrame. It checks for missing values, duplicates, data types, numeric ranges, and impossible or outlier values.

# Key Checks Performed:

# Dataset shape – rows and columns.

# Missing values – column-wise count.

# Duplicate rows – total duplicates in the dataset.

# Data types – helps ensure correct types for analysis.

# Patient ID uniqueness – checks for repeated patients.

# Numeric summary – min, max, mean values of numeric columns.

# Time range – earliest and latest timestamps.

# Impossible/outlier values – negative values or unrealistic glucose readings.

# Value ranges – ensures columns like glucose, steps, heart rate, and calories fall in reasonable ranges

In [30]:
#Data Quality Report for demographic data
data_quality_report_common(demographic_df)
demographic_data_quality_report(demographic_df)
print("DATA QUALITY REPORT Completed")


DATA QUALITY REPORT
Rows: 25, Columns: 7
**************************************************

 Check for Missing Values:
**************************************************

 Number of missing column values are:
 patient_id                       0
age                              0
gender                           0
race                             0
avg_sleep_duration_hrs           0
sleep_quality_score              0
sleep_disturbances_percentage    0
dtype: int64  
**************************************************

 Check for Duplicate Rows:
**************************************************

 Number of duplicate rows  are : 0
**************************************************

 Check for Column Data Types:
**************************************************
patient_id                        object
age                                int64
gender                            object
race                              object
avg_sleep_duration_hrs           float64
sleep_quality_score      

In [35]:
#Data Quality Report for merged all patients and demographic data
data_quality_report_common(patients_demographic_df)
demographic_data_quality_report(patients_demographic_df)
print("DATA QUALITY REPORT Completed")


DATA QUALITY REPORT
Rows: 309392, Columns: 15
**************************************************

 Check for Missing Values:
**************************************************

 Number of missing column values are:
 time                             0
glucose                          0
calories                         0
heart_rate                       0
steps                            0
basal_rate                       0
bolus_volume_delivered           0
carb_input                       0
patient_id                       0
age                              0
gender                           0
race                             0
avg_sleep_duration_hrs           0
sleep_quality_score              0
sleep_disturbances_percentage    0
dtype: int64  
**************************************************

 Check for Duplicate Rows:
**************************************************

 Number of duplicate rows  are : 0
**************************************************

 Check for Column Data Typ

This is comment box
